# Preprocessing
This notebook contains all code for the preprocessing of the KDD Cup 98 datasets.
* Splits into learning and test
* Learns the transformation pipeline on the learning dataset for future use
* Prepares the data for model fitting

This will be done with scikit-learn's transforming framework in order to ensure all transformations are applied identically on training, test and validation datasets.


The transformations are 'learned' on the training dataset and then applied to the test dataset and new data later on.

In [2]:
%load_ext autoreload

In [4]:
%autoreload 2
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Change to project root
if not os.getcwd()[-4:] == 'code':
    os.chdir("../code")
    
# Load custom code
import kdd98.data_loader as dl
import kdd98.utils_transformer as ut
from kdd98.transformers import *
from kdd98.config import App

In [3]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
# seaborn config
import seaborn as sns
sns.set(color_codes=True)
sns.set_style('ticks')
sns.axes_style({'spines.right': False,
                'axes.spines.top': False})
sns.set_palette(App.config("color_palette"))
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.figsize'] = (12, 8)

# figures:
# Where to save the figures
PROJECT_ROOT_DIR = "../../"
CHAPTER_ID = "preprocessing"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "figures", CHAPTER_ID)

if not os.path.exists(IMAGES_PATH):
    os.makedirs(IMAGES_PATH)


def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Loading the learning dataset


Set working directory to main code folder

In [4]:
data_loader = dl.KDD98DataLoader("cup98LRN.txt")
learning_raw = data_loader.get_dataset()

## Splitting into training- and test dataset

Before applying *any* transformations, the dataset will be split 80/20 into a learning and test set.

Let's look at feature TARGET_B, which describes whether a person has donated or not:

In [5]:
learning_raw.TARGET_B.value_counts(normalize=True) # 5 % of recipients have donated.

0    0.949241
1    0.050759
Name: TARGET_B, dtype: float64

We want to preserve this ratio in the split datasets. scikit-learn provides a method for achieving this.

In [6]:
from config import App
seed = App.config("random_seed")
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, train_size=0.8, random_state=seed)
for learn_index, test_index in splitter.split(learning_raw, learning_raw.TARGET_B.astype('int')):
    l_i = learn_index
    t_i = test_index
    kdd_learn = learning_raw.iloc[learn_index]
    kdd_test = learning_raw.iloc[test_index]
    

Now, check that the two sets are really disjoint

In [7]:
set(kdd_learn.index).intersection(kdd_test.index)

set()

Check the frequencies of the donors in the sets:

In [8]:
kdd_learn['TARGET_B'].value_counts(normalize=True)

0    0.949246
1    0.050754
Name: TARGET_B, dtype: float64

In [9]:
kdd_test['TARGET_B'].value_counts(normalize=True)

0    0.949222
1    0.050778
Name: TARGET_B, dtype: float64

## Separating features and label

First, we separate the features from the labels. We will also remove the label "TARGET_B", which is an indicator variable for donors that is no longer of interest

**All preprocessing is performed on *kdd_learn_feat***

In [31]:
kdd_learn_feat = kdd_learn.drop(['TARGET_B', 'TARGET_D'],axis=1).copy()
kdd_learn_labels = kdd_learn[['TARGET_B','TARGET_D']].copy()

## Cleaning pipeline

The cleaning pipeline results in a dataset with numerical (binary features encoded correclty), categorial and string date features.

Following this step, feature extraction, imputation, dropping of constant and sparse features and ensuring all data is numerical can be tackled.

### Automating cleaning and feature extraction

The function below allows for a one-step cleaning and feature extraction, returning a pandas dataframe with all features correctly labelled.

As has been seen in EDA, some features are redundant. These will not be processed by the pipeline here.

In [32]:
preproc_pipe = Pipeline([
    ("binary_features",
     ColumnTransformer([
        ("binary_x_bl",
         BinaryFeatureRecode(value_map={'true': 'X', 'false': ' '}, correct_noisy=False),
         ['PEPSTRFL', 'NOEXCH', 'MAJOR', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP']
         ),
        ("binary_y_n",
         BinaryFeatureRecode(value_map={'true': 'Y', 'false': 'N'}, correct_noisy=False),
         ['COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO',
          'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN',  'BOATS', 'WALKER', 'KIDSTUFF',
          'CARDS', 'PLATES']
         ),
        ("binary_e_i",
         BinaryFeatureRecode(value_map={'true': "E", 'false': 'I'}, correct_noisy=False),
         ['AGEFLAG']
         ),
        ("binary_h_u",
         BinaryFeatureRecode(value_map={'true': "H", 'false': 'U'}, correct_noisy=False),
         ['HOMEOWNR']),
        ("binary_b_bl",
         BinaryFeatureRecode(value_map={'true': 'B', 'false': ' '}, correct_noisy=False),
         ['MAILCODE']
         ),
        ("binary_1_0",
         BinaryFeatureRecode(value_map={'true': '1', 'false': '0'}, correct_noisy=False),
         ['HPHONE_D']
         )
        ])
    ),
    ("date_features",
     ColumnTransformer([
        ("months_to_donation", MonthsToDonation(), dl.promo_history_dates+dl.giving_history_dates),
         ("time_last_donation", DeltaTime(unit='months'), ['LASTDATE','MINRDATE','MAXRDATE','MAXADATE']),
        ("membership_years", DeltaTime(unit='years'),['ODATEDW'])
        ])
    ),
    ("osource",
      ColumnTransformer([("hash_osource", HashingEncoder(), ['OSOURCE'])])
    ),
    ("tcode",
      ColumnTransformer([("hash_tcode", HashingEncoder(), ['TCODE'])])
    ),
    ("zip",
      ColumnTransformer([("hash_zip", HashingEncoder(), ['ZIP'])])
    ),
    ("rfa",
      Pipeline([
        ("spread_rfa", ColumnTransformer([('spread', MultiByteExtract(["R", "F", "A"]), dl.nominal_features[2:])])),
        ("order_multibytes", OrdinalEncoder(mapping=dl.ordinal_mapping_rfa,handle_unknown='ignore'))
      ])
    ),
    ("domain",
     Pipeline([
         ("spread_domain", ColumnTransformer([("spread",MultiByteExtract(["Urbanicity", "SocioEconomic"]),["DOMAIN"])])),
         ("recode_socioecon", RecodeUrbanSocioEconomic())
     ])
    ),
    ("mdmaud",
     ColumnTransformer([
         ("mdmaud",
         OrdinalEncoder(mapping=dl.ordinal_mapping_mdmaud,handle_unknown='ignore'),
         ['MDMAUD_R','MDMAUD_A'])
     ])
    )
])

In [33]:
preproc_pipe.fit(kdd_learn_feat)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [23]:
def do_cleaning_preprocessing(dataset, fit=False):
    
    # Cleaning stage
    
    # Binary features
    binary_transformers = ColumnTransformer([
        ("binary_x_bl",
         BinaryFeatureRecode(value_map={'true': 'X', 'false': ' '}, correct_noisy=False),
         ['PEPSTRFL', 'NOEXCH', 'MAJOR', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP']
         ),
        ("binary_y_n",
         BinaryFeatureRecode(value_map={'true': 'Y', 'false': 'N'}, correct_noisy=False),
         ['COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO',
          'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN',  'BOATS', 'WALKER', 'KIDSTUFF',
          'CARDS', 'PLATES']
         ),
        ("binary_e_i",
         BinaryFeatureRecode(value_map={'true': "E", 'false': 'I'}, correct_noisy=False),
         ['AGEFLAG']
         ),
        ("binary_h_u",
         BinaryFeatureRecode(value_map={'true': "H", 'false': 'U'}, correct_noisy=False),
         ['HOMEOWNR']),
        ("binary_b_bl",
         BinaryFeatureRecode(value_map={'true': 'B', 'false': ' '}, correct_noisy=False),
         ['MAILCODE']
         ),
        ("binary_1_0",
         BinaryFeatureRecode(value_map={'true': '1', 'false': '0'}, correct_noisy=False),
         ['HPHONE_D']
         )
    ])
    
    
    # Dates
    don_hist_transformer = ColumnTransformer([
        ("months_to_donation",
         MonthsToDonation(),
         dl.promo_history_dates+dl.giving_history_dates
         )
    ])
    
    
    timedelta_transformer = ColumnTransformer([
        ("time_last_donation", DeltaTime(unit='months'), ['LASTDATE','MINRDATE','MAXRDATE','MAXADATE']),
        ("membership_years", DeltaTime(unit='years'),['ODATEDW'])
    ])
    
    
    # Categorical Features
    
    # Nominals
    osource_transformer = ColumnTransformer([
        ("hash_osource", HashingEncoder(), ['OSOURCE'])
    ])
    
    tcode_transformer = ColumnTransformer([
        ("hash_tcode", HashingEncoder(), ['TCODE'])
    ])
    
    # Ordinals
    multibyte_transformer = ColumnTransformer([
        ("spread",
         MultiByteExtract(["R", "F", "A"]),
         dl.nominal_features[2:])
    ])
    
    domain_transformer = Pipeline([
        ("spread", ColumnTransformer([
            ("spread_domain",
            MultiByteExtract(["Urbanicity", "SocioEconomic"]),
            ["DOMAIN"])
        ])),
        ("recode_socioecon", RecodeUrbanSocioEconomic())
    ])

    
    # Remaining ordinals
    ordinal_transformer = ColumnTransformer([
        ("order_ordinals",
        OrdinalEncoder(mapping=dl.ordinal_mapping_mdmaud,handle_unknown='ignore'),
        ['MDMAUD_R','MDMAUD_A']),
        ("order_multibytes",
        OrdinalEncoder(mapping=dl.ordinal_mapping_rfa,handle_unknown='ignore'),
         list(dataset.filter(like="RFA_",axis=1).columns))
    ])
    
    # Transforming the data (possibly fitting first) and rebuilding the pandas dataframe
    
    binarys = binary_transformers.fit_transform(dataset)
    dataset = ut.update_df_with_transformed(dataset, binarys, binary_transformers)
    donation_responses = don_hist_transformer.fit_transform(dataset)
    dataset = ut.update_df_with_transformed(dataset, donation_responses, don_hist_transformer)
    timedeltas = timedelta_transformer.fit_transform(dataset)
    dataset = ut.update_df_with_transformed(dataset, timedeltas, timedelta_transformer, drop=dl.date_features)
    osource = osource_transformer.fit_transform(dataset)
    dataset = ut.update_df_with_transformed(dataset, osource, osource_transformer)
    tcode = tcode_transformer.fit_transform(dataset)
    dataset = ut.update_df_with_transformed(dataset, tcode, osource_transformer)
    multibytes = multibyte_transformer.fit_transform(dataset)
    dataset = ut.update_df_with_transformed(dataset, multibytes, multibyte_transformer, drop=dl.nominal_features, new_dtype="category")
    domains = domain_transformer.fit_transform(dataset)
    dataset = ut.update_df_with_transformed(dataset, domains, domain_transformer)
    ordinals = ordinal_transformer.fit_transform(dataset)
    dataset = ut.update_df_with_transformed(dataset, ordinals, ordinal_transformer)
    
    return dataset

In [ ]:
preprocessing_pipe = Pipeline([
    
])

In [24]:
dataset = do_cleaning_preprocessing(kdd_learn_feat)

NameError: name 'Pipeline' is not defined

The next step is to one-hot encode categorical features. This results in an all-numeric dataframe.

In [ ]:
dataset2 = pd.get_dummies(dataset)

In [ ]:
dataset2.info()

## Imputation of missing values

https://github.com/epsilon-machine/missingpy

Olga Troyanskaya, Michael Cantor, Gavin Sherlock, Pat Brown, Trevor Hastie, Robert Tibshirani, David Botstein and Russ B. Altman, Missing value estimation methods for DNA microarrays, BIOINFORMATICS Vol. 17 no. 6, 2001 Pages 520-525

This step requires that we first drop features with more than 80% missing values for the KNNImputer to work.

In [ ]:
[c for c in dataset2.columns if dataset2[c].count() / len(dataset2.index) <= 0.2]

In [ ]:
dataset2.drop([c for c in dataset2.columns if dataset2[c].count() / len(dataset2.index) <= 0.2],axis=1,inplace=True)

We set weights to distance so that binary and categorical features get an integer value:
https://www.queryxchange.com/q/27_52658127/imputing-missing-values-with-knn/

In [ ]:
from missingpy import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="distance")
kdd_learn_feat_imputed = imputer.fit_transform(dataset2)

## Removing constant features

As per the documentation, features with either low variance or very few non-NA examples are to be dropped.

In [ ]:
[c for c in kdd_learn_feat_imputed.columns if kdd_learn_feat_imputed[c].var() <= 1e-5]

In [ ]:
def get_low_variance_cols(df=None, cols=None,
                             skip_cols=[], thresh=1e-5,
                             autoremove=False):
    """
    Wrapper for sklearn VarianceThreshold for use on pandas dataframes.
    """
    try:
        # get list of all the original df cols
        all_cols = df.select_dtypes(include="number").columns

        # remove `skip_cols`
        remaining_cols = all_cols.drop(skip_cols)

        # get length of new index
        max_index = len(remaining_cols) - 1

        # get indices for `skip_cols`
        skipped_idx = [all_cols.get_loc(column)
                       for column
                       in skip_cols]

        # adjust insert location by the number of cols removed
        # (for non-zero insertion locations) to keep relative
        # locations intact
        for idx, item in enumerate(skipped_idx):
            if item > max_index:
                diff = item - max_index
                skipped_idx[idx] -= diff
            if item == max_index:
                diff = item - len(skip_cols)
                skipped_idx[idx] -= diff
            if idx == 0:
                skipped_idx[idx] = item

        # get values of `skip_cols`
        skipped_values = df.iloc[:, skipped_idx].values

        # get dataframe values
        X = df.loc[:, remaining_cols].values

        # instantiate VarianceThreshold object
        vt = VarianceThreshold(threshold=thresh)

        # fit vt to data
        vt.fit(X)

        # get the indices of the features that are being kept
        feature_indices = vt.get_support(indices=True)

        # remove low-variance cols from index
        feature_names = [remaining_cols[idx]
                         for idx, _
                         in enumerate(remaining_cols)
                         if idx
                         in feature_indices]

        # get the cols to be removed
        removed_features = list(np.setdiff1d(remaining_cols,
                                             feature_names))
        print("Found {0} low-variance cols."
              .format(len(removed_features)))

        # remove the cols
        if autoremove:
            print("Removing low-variance features.")
            # remove the low-variance cols
            X_removed = vt.transform(X)

            print("Reassembling the dataframe (with low-variance "
                  "features removed).")
            # re-assemble the dataframe
            df = pd.DataFrame(data=X_removed,
                                  cols=feature_names)

            # add back the `skip_cols`
            for idx, index in enumerate(skipped_idx):
                df.insert(loc=index,
                              column=skip_cols[idx],
                              value=skipped_values[:, idx])
            print("Succesfully removed low-variance cols.")

        # do not remove cols
        else:
            print("No changes have been made to the dataframe.")

    except Exception as e:
        print(e)
        print("Could not remove low-variance features. Something "
              "went wrong.")
        pass

    return df, removed_features

In [ ]:
df, removed = get_low_variance_cols(kdd_learn_feat_2)

## Exploring strategies for specific feature types

* Noisy data: Correction of data entry / formatting errors
    - These errors must be corrected without excluding the records in question
* Missing data: Has to be inferred from known values
    - (e.g., mean, median, mode, a modeled value).
    - One exception to this rule is the attributes containing 99.5 percent or more missings. These are to be dropped
* Sparse data: Events actually represented in given data make only a very small subset of the event space are to be dropped
* Constant values are to be dropped

### Constant and Sparse Features

Features where only one value is present and those where the majority is empty are to be dropped.


In [ ]:
const_sparse_transformer = DropSparseLowVar(keep_anyways=["RAMNT_\d{1,2}", "MONTHS_TO_DONATION_\d{1,2}"])
cs = const_sparse_transformer.fit(learning)
cs = const_sparse_transformer.fit_transform(learning)
set(cs.columns)
const_sparse_transformer.get_feature_names()

### Numerical features

In [ ]:
numerical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

### Categorical features

### Remaining object features

In [ ]:
objects = learning_raw.select_dtypes(include='object').columns
print(objects)

In [ ]:
for f in objects:
    print(f+": "+learning_raw[f].unique())

These are two types:

* ZIP: Malformed zip codes. Some have a dash at the end, which has to be removed.
* Multibyte values. These can be extracted into separate features bytewise. However, this is done in feature extraction later on

## Preprocessing Pipeline

It is now time to construct the preprocessing pipeline. A set of transforming operations is concatenated to a sequence of operations. This pipeline is the learned on the learning dataset. All transformations to the learning dataset will then later be applied to the test dataset and to new data.

In [ ]:
numerical_feats = list(kdd_learn_feat.select_dtypes(include=np.number).columns)
categorical_feats = list(kdd_learn_feat.select_dtypes(include=np.number).columns)

With all categories now properly formatted, it is time for one-hot encoding. The sklearn pipeline also has an impute transformation. NaN's get their own level, "missing". This step results in a huge increase in the dimension of the feature space. It is also heavy on computation.

In [ ]:
cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("one_hot",  OneHotEncoder(impute_missing=True,use_cat_names=True,return_df=True))
])

categories_transformer = ColumnTransformer([
    ("cat_encoder",
     cat_pipe,
     list(kdd_learn_feat.select_dtypes(include="category").columns))
])

Interests and donations

In [ ]:
data = learning_raw.loc[:,dl.interest_features+["TARGET_D"]].fillna(0)
interests = pd.melt(data,value_vars=dl.interest_features, value_name="Interest")
data.head()

Features with constant values:

## Feature Selection
Meant to reduce dimensionality by selecting only features that are 'interesting enough' to be considered in order to boost performance of calculations / improve accuracy of the estimator
- By variance threshold
- Recursive Feature Elimination by Cross-Validation
- L1-based feature selection (Logistic Regression, Lasso, SVM)
- Tree-based feature selection

See [scikit-learn: feature selection](http://scikit-learn.org/stable/modules/feature_selection.html#feature-selection)

### Removing constant features (zero variance)

sklearn.feature_selection_variance_threshold

In [ ]:
for column in learning.columns:
        if len(learning[column].unique()) == 1:
            print(column)

### Sparse Features

In [ ]:
sparse_features = []
for column in learning:
    top_freq = learning[column].value_counts(normalize=True).iloc[0]
    if top_freq > 0.995:
        sparse_features.append(column)
        print(column+" has a top frequency of: " + str(top_freq))
        print(learning[column].value_counts(normalize=True))

In [ ]:
sparse_features

### Advanced approaches

* If overfitting is a problem, ensemble-learning or tree learning can be used to find important features, then apply SelectFromModel before the actual estimator. See http://scikit-learn.org/stable/modules/feature_selection.html

## Feature Extraction
All explanatory fields have to be numerical for the subsequent operations with scikit-learn. Here, the necessary feature extractions are performed.

See [scikit-learn: feature extraction](http://scikit-learn.org/stable/modules/feature_extraction.html)

In [ ]:
import pandas as pd

In [ ]:
symbolic_features = []
symbolic_features.append(tds.SymbolicFeatureSpreader(
    "DOMAIN", ["U", "S"])) #Urbanicity, SocioEconomicStatus
# RFA_2 is already spread out
for i in range(3, 25):
    feature = "_".join(["RFA", str(i)])
    symbolic_features.append(tds.SymbolicFeatureSpreader(
        feature, ["R", "F", "A"])) # Recency, Frequency, Amount

spread_multibyte = pd.DataFrame(index=learning_raw.index)
for f in symbolic_features:
    f.set_tidy_dataset_ref(learning_raw)
    spread_multibyte = pd.concat([spread_multibyte,f.spread(inplace=False)],axis=1)

In [ ]:
spread_multibyte.info()

# PCA

A first look at important features

In [ ]:
from sklearn import decomposition

In [ ]:
X = learning.drop(["TARGET_B","TARGET_D"],axis=1)

In [ ]:
n_comp = 3
pca = decomposition.PCA(n_components = n_comp)
pca.fit(X)
result = pd.DataFrame(pca.transform(X), columns=["PCA%i" % i for i in range(n_comp)], index=X.index)

In [ ]:
import cProfile
domain_spreader = tds.SymbolicFieldToDummies(learning,"RFA_24",["Recency", "Frequency", "Amount"])
cProfile.run('domain_spreader.spread()', sort='time')

In [ ]:
learning.head()

In [ ]:
import os
import numpy as np
import sys
os.getcwd()
proj_dir = os.path.split(os.getcwd())[0]
if proj_dir not in sys.path:
    sys.path.append(proj_dir)

In [ ]:
import eda.tidy_dataset as tds
tidy = tds.TidyDataset("cup98LRN.txt")

In [ ]:
raw = tidy.get_raw_data()

In [ ]:
spreader = tds.SymbolicFieldToDummies(
    raw, "RFA_24", ["Recency", "Frequency", "Amount"])
spreader.spread()